# 민원 현황 세부 분석

# 0. 들어가기 전

본 분석에서는 EDA 및 토픽모델링을 활용한 민원 현황 분석 및 시각화를 진행하였습니다.  
EDA로 대략적인 분석 과정을 잡아가려고 했고, 토픽모델링을 통해 주어진 데이터로 확인하기 힘든 주요 민원 주제들을 확인하고자 하였습니다.    
또한 기존의 워드클라우드나 단어 빈도수 기반의 키워드 분석을 지양하고자 하였습니다.



> 드라이브 마운트

In [35]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


> 분석에 필요한 라이브러리 설치 및 업데이트

In [36]:
!pip install --upgrade plotly --quiet #시각화 툴
!pip install soynlp --quiet #단어사전 구축용
!pip install kiwipiepy --quiet #형태소 분석기

> 라이브러리 불러오기

In [37]:
import os
import pandas as pd
import numpy as np
import warnings

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

from soynlp.noun import LRNounExtractor_v2
from kiwipiepy import Kiwi, Option
from sklearn.feature_extraction.text import TfidfVectorizer

warnings.filterwarnings("ignore")

os.chdir('/gdrive/MyDrive/민원분석/')
dir = os.getcwd()

In [38]:
class plot_type:
    def __init__(self, data):
        self.data = data
        self.fig = None
        self.update_layout = None
    
    def count_percentage(self):
        if 'counts' in self.data.columns:
            self.data['percentage'] = round(self.data['counts'] / self.data['counts'].sum() * 100, 2)
    
    def bar(self, x, y, **kwargs):
        plot_type.count_percentage(self)

        if 'percentage' in self.data.columns:
            hover_data = ['percentage']
        else:
            hover_data = None

        self.fig = px.bar(self.data, x=x, y=y, hover_data=hover_data, **kwargs)

    def pie(self, values, names, **kwargs):
        self.fig = px.pie(self.data, values=values, names=names, **kwargs)

    def plot(self):
        self.fig.show()

In [106]:
%%time
root = pd.read_parquet(dir + '/Data/minwon_data.parquet')

CPU times: user 630 ms, sys: 243 ms, total: 873 ms
Wall time: 800 ms


# 1. EDA 및 시각화

> 모든 시각화 그래프들은 Interactive하게 만들었습니다.  
마우스를 위에 올리면 자세한 값들을 확인할 수 있습니다.

## 새로운 변수 생성

In [107]:
root.columns

Index(['제목', '질문내용', '답변내용', '등록일시', '최상위기관명', '분야코드', '분야명', '담당부서명'], dtype='object')

제공받은 데이터의 Column은 `제목`, `질문내용`, `답변내용`, `등록일시`, `최상위기관명`, `분야코드`, `분야명`, `담당부서명`으로, 분석에 들어가기에 앞서 각 Column들의 내용을 확인하였다.  
그리고 `등록일시`의 데이터를 연도, 월, 일 등으로 쪼개어 EDA를 수행해보고자 하였다.  

In [108]:
root['등록일시'] = pd.to_datetime(root['등록일시'])

root['등록년도'] = root['등록일시'].dt.year#.astype(int)
root['등록월']   = root['등록일시'].dt.month#.astype(int)
root['등록일']   = root['등록일시'].dt.day#.astype(int)
root['등록요일'] = root['등록일시'].dt.day_name()
root['등록시간'] = root['등록일시'].dt.hour#.astype(int)
root['오전오후'] = pd.Series(np.where(root['등록시간'].to_numpy() > 12, 'PM', 'AM'))

In [109]:
root['최상위기관명'].unique()

array(['경상북도 의성군', '전라북도 순창군', '전라남도 순천시', '경상북도 봉화군', '전라남도 광양시',
       '전라북도 무주군', '전라남도 영광군', '서울특별시', '전라북도 정읍시', '강원도', '경상남도 고성군',
       '충청북도 영동군', '충청북도 음성군', '서울특별시 동대문구', '경기도 의정부시', '경기도 가평군',
       '경기도 의왕시', '서울특별시 강동구', '경기도 안양시', '인천광역시 미추홀구', '광주광역시 북구',
       '대구광역시 북구', '경기도 이천시', '경기도 하남시', '광주광역시 남구', '충청남도 천안시', '해양경찰청',
       '소방청', '전라남도 목포시', '경찰청', '부산광역시 사하구', '부산광역시 동래구', '경상남도 산청군',
       '전라북도 전주시', '충청남도 공주시', '경상남도 사천시', '충청남도 서산시', '경상북도 경주시',
       '충청남도 아산시', '경상남도 김해시', '전라북도 남원시', '경상남도 남해군', '부산광역시', '대구광역시',
       '제주특별자치도', '대구광역시 동구', '서울특별시 은평구', '부산광역시 금정구', '부산광역시 남구',
       '전라북도 김제시', '광주광역시', '경상남도 함안군', '서울특별시 성동구', '서울특별시 강서구',
       '부산광역시 부산진구', '충청북도 옥천군', '경기도 안성시', '광주광역시 광산구', '충청북도 보은군',
       '충청북도 괴산군', '충청남도 금산군', '서울특별시 강남구', '경상북도 영주시', '경상북도 경산시',
       '서울특별시 종로구', '서울특별시 용산구', '서울특별시 서대문구', '충청북도', '전라남도 신안군',
       '경상북도 예천군', '전라남도 구례군', '부산광역시 수영구', '서울특별시 관악구', '통계청',
       '경상남도 의령군', '충청남도', '경상남도 거제시', 

`최상위기관명`은 `등록일시`와 반대로 여러개로 통합하여 EDA를 수행하고자 하였다.  
최상위기관명을 확인해보니 크게 서울특별시 ~구, 경상북도 ~군 등의 지방청사와 국세청, 국방부 등의 중앙행정기관 그리고 한국산업인력공단과 같은 공공기관으로 분류할 수 있을 것 같았다.   

In [110]:
%%time

conditions = {
    '중앙행정기관' : root['최상위기관명'].str.endswith(('부', '처', '청', '회')),
    '광역자치단체' : root['최상위기관명'].str.endswith(('특별시', '도', '자치시')),
    '기초자치단체' : root['최상위기관명'].str.endswith(('시', '군', '구'))
}

root['기관구분'] = (
    np.select(condlist=conditions.values(), choicelist=conditions.keys(), default='공공기관')
    )

CPU times: user 70.9 ms, sys: 0 ns, total: 70.9 ms
Wall time: 74.2 ms


그리고 추가로 각 지방청사들은 지역명으로 분류하여 새로운 column을 만들어주었다.  

In [111]:
%%time

conditions = {
    '경상도' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('경상')),
    '전라도' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('전라')),
    '충청도' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('충청')),
    '강원도' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('강원')),
    '경기도' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('경기')),
    '서울특별시' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('서울')),
    '인천광역시' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('인천')),
    '부산광역시' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('부산')),
    '광주광역시' :(root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('광주')),
    '대구광역시' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('대구')),
    '울산광역시' : (root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('울산')),
    '대전광역시' :(root['기관구분'].isin(['광역자치단체', '기초자치단체'])) & (root['최상위기관명'].str.contains('대전')),
}

root['지역'] = (
    np.select(condlist=conditions.values(), choicelist=conditions.keys(), default='중앙행정기관')
    )

CPU times: user 290 ms, sys: 0 ns, total: 290 ms
Wall time: 292 ms


## 연도별 민원 수

In [112]:
years = root.groupby(by=["등록년도"]).size().reset_index(name="counts").dropna().sort_values('등록년도', ascending=False)
years['등록년도'] = years['등록년도'].astype(int)

bar_years = plot_type(years)
bar_years.bar(x='등록년도', y='counts')
bar_years.fig.update_layout(
    title = '연도별 민원 수',
    font = {'size' : 15,
            'family' : 'NanumGothic ExtraBold'}
)

bar_years.plot()

**민원데이터는 2020년과 2021년의 데이터가 약 93%정도를 차지하였습니다**  
**이 민원 데이터는 2021년의 내용이 압도적으로 많기 때문에 이후 분석 내용은 2021년을 타겟으로 해야겠다고 생각했습니다. 그러나 아직 2021년이 마무리되지 않았기 때문에 월별 비교 등을 수행하기 위해 2020년 역시 데이터 양은 2021년에 비해 작지만 포함하였습니다.**

**또한 나머지를 제거해도 약 7%정도이기 때문에 분석에 영향이 거의 없을것이라 판단하였고 이후 진행되는 분석에서는 2020년 이전 데이터들은 모두 제거하고 사용하였습니다.**  

In [113]:
print(f'Before remove : {root.shape}')
root = root.loc[root['등록년도'] >= 2020]
print(f'After remove : {root.shape}') #등록년도가 NaN값인것까지 함께 제거됨

Before remove : (57249, 16)
After remove : (47328, 16)


## 기관별 민원 수

In [114]:
dep_cat = root.groupby(by=["기관구분"]).size().reset_index(name="counts")
bar_dep_cat = plot_type(dep_cat)
bar_dep_cat.pie('counts', '기관구분', hole=0.3)
bar_dep_cat.fig.update_traces(textposition='inside', textinfo='percent+label')
bar_dep_cat.fig.update_layout(
    title = '기관별 민원 수',
    font = {'size' : 15,
            'family' : 'NanumGothic ExtraBold'}
)
bar_dep_cat.plot()

**중앙행정기관으로 접수된 민원들이 약 66%로 가장 많았고 그 다음으로 기초자치단체 약 31%, 광역자치단체 2%, 공공기관 0.2% 를 차지하였다는것을 확인할 수 있습니다.**  

**수집된 2021년,2020년의 민원은 약 97%가 중앙행정기관 및 기초자치단체에 접수되었고, 절반이 넘는 민원이 중앙행정기관에 접수되었기 때문에 이 민원은 기존 구청, 시청 등에 접수되는 민원과 내용이 다를 것이라는 생각이 들었습니다.**  

**그래서 우선 최상위기관에 접수된 민원 수 상위 10개를 파악하고자 하였습니다.**

## 최상위기관별 민원 수

In [121]:
dep_name = root.groupby(by=["최상위기관명"]).size().reset_index(name="counts").sort_values('counts', ascending=False).head(10)

bar_dep_name = plot_type(dep_name)
bar_dep_name.bar(x='최상위기관명', y='counts')
bar_dep_name.fig.update_layout(
    title = '최상위기관별 민원 수',
    font = {'size' : 15,
            'family' : 'NanumGothic ExtraBold'}
)
bar_dep_name.plot()

**접수된 민원 수가 가장 많은 부서는 해양수산부-국세청-서울특별시교육청-부산광역시교육청-교육부 순으로 특이하게 해양수산부가 다른 기관들에 비해 접수된 민원이 많은 것을 확인할 수 있었습니다.**  

**그래서 각 기관별로 어떤 민원이 접수되었는지 궁금증이 생겼습니다.**

>교욱청에 접수된 민원은 교육관련, 국세청은 세금납부관련 문의가 많을까?  
>해양수산부에 접수되는 해양관련문의는 무슨 문의일까?

## 분야별 민원 수

In [122]:
cat_name = root.groupby(by=["분야명"]).size().reset_index(name="counts")

bar_cat_name = plot_type(cat_name)
# bar_cat_name.bar(x='분야명', y='counts', color='분야명')
# bar_cat_name.fig.update_layout(xaxis_categoryorder='total descending')
# #bar_cat_name.fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
# bar_cat_name.plot()

In [126]:
bar_cat_name.pie('counts', '분야명', hole=0.3)
bar_cat_name.fig.update_traces(textposition='inside', textinfo='percent+label')
bar_cat_name.fig.update_layout(
    title = '기관별 민원 수',
    font = {'size' : 13,
            'family' : 'NanumGothic ExtraBold'}
)
bar_cat_name.plot()

**접수 분야는 행정/자치/안전(38.3%) - 교육/문화/체육/관광(23.1%) - 국토/교통/농림/해양(14.8%) - 재정/금융/소비자(9.71%) 순이었습니다.**    

**중앙행정기관보다는 자치단체의 행정/자치/안전 분야의 민원이 비중이 클 것이고 다른 분야들은 중앙행정기관에 접수되지 않았을까 하는 가설을 세웠습니다.**  
**그래서 위 내용을 조금 더 자세히 보기 위해 위에서 분류했던 중앙행정기관, 자치단체 별로 분야에 차이가 있는지 확인하고자 하였습니다.**  


## 기관별로 집계된 분야별 민원 수

In [55]:
#dep_category = root.groupby(['기관구분','분야명']).size().reset_index(name='counts')

In [56]:
dep_category = root.groupby(['기관구분','분야명']).size().reset_index(name='counts')
bar1 = plot_type(dep_category)
bar1.bar(x='분야명', y='counts', 
         color='기관구분', facet_col='기관구분', color_discrete_sequence=px.colors.qualitative.Plotly)

bar1.fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
#bar1.fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
bar1.plot()

**기관마다 분야별 민원 수를 확인해보니 자치단체에서는 행정/자치/안전뿐이었고, 중앙행정기관은 교육/문화/체육/관광, 국토/교통/농림/해양, 재정/금융/소비자가 많다는 것을 알 수 있었습니다.**  
**위의 가설이 맞다는 것을 확인할 수 있었고, 분야별 민원수에서 1위를 차지했던 행정/자치/안전은 사실 자치단체에 약 90%정도 집중되어있었음을 알 수 있었습니다.**  

**그리고 이후 분석시 행정/자치/안전은 기초자치단체의 행정/자치/안전 분야를 분석하고, 그 외 다른 분야는 중앙행정기관을 분석하는 것으로 진행방향을 잡을 수 있었습니다.**   
> 자치단체의 행정/자치/안전 민원과 중앙기관의 행정/자치/안전의 내용이 다를것이라 생각하였습니다.

## 월별 민원 수

**앞선 연도별 민원 수 분석에서 2021년의 민원 수 비중이 매우 컸던 것을 확인할 수 있었습니다.**  
**단순히 합계로 월별 민원을 분석한다면, 예를 들어 12월은 2021년에는 포함되지 않았기 때문에 당연히 민원 수가 적게 나올것입니다.**  
**따라서 월별 민원수를 분석할때는 비중을 비슷하게 맞춰주기 위해서 연도별 총계값을 나눠주는 간단한 정규화를 수행하였습니다.**

In [57]:
from plotly.subplots import make_subplots

In [58]:
months = root.groupby(by=['등록년도','등록월', '분야명']).size().reset_index(name='counts').dropna().sort_values('등록월', ascending=False)
test   = root.groupby(by=['등록월']).size().reset_index(name='counts').sort_values('등록월')
# tmp_ = months.loc[months['등록월']==12]
# months = months.append(tmp_).reset_index(drop=True) # 그림그릴떄 시작점 끝점 맞춰주는 용도

In [59]:
# data_ = test
# r = 'counts'
# theta = data_['등록월'].to_numpy() * 30 # degree

# lines_test = Polar(data_)
# lines_test.line(r='counts', theta=theta)
# lines_test.fig.update_traces(fill='toself')
# lines_test.fig.update_traces(mode='markers+lines')
# lines_test.fig.update_layout(
#         polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
#         polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
#         #polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
#         title='정규화를 수행하지 않은 월별 등록민원 수',
#         title_x=0.5,
#         title_font_size=20
#     )


# data_ = months
# r = 'counts'
# theta = data_['등록월'].to_numpy() * 30 # degree

# lines = Polar(data_)
# lines.line(r='counts', theta=theta)
# lines.fig.update_traces(fill='toself')
# lines.fig.update_traces(mode='markers+lines')
# lines.fig.update_layout(
#         polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
#         polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
#         #polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
#         title='정규화를 수행하지 않은 월별 등록민원 수',
#         title_x=0.5,
#         title_font_size=20
#     );

In [60]:
# sub = make_subplots(rows=1, cols=2)

# sub.add_trace(
#     lines_test.fig,
#     row=1, col=1
# )

# sub.add_trace(
#     lines.fig,
#     row=1, col=2
# )

# sub.update_layout(height=600, width=800, title_text="Side By Side Subplots")
# sub.show()

# # data_ = test
# # r = 'counts'
# # theta = data_['등록월'].to_numpy() * 30 # degree

# # lines_test = Polar(data_)
# # lines_test.line(r='counts', theta=theta)
# # lines_test.fig.update_traces(fill='toself')
# # lines_test.fig.update_traces(mode='markers+lines')
# # lines_test.fig.update_layout(
# #         polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
# #         polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
# #         #polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
# #         title='정규화를 수행하지 않은 월별 등록민원 수',
# #         title_x=0.5,
# #         title_font_size=20
# #     )

In [61]:
year_20 = months.loc[months['등록년도']==2020]
year_20['counts'] = year_20['counts'] / sum(year_20['counts'])

year_21 = months.loc[months['등록년도']==2021]
year_21['counts'] = year_21['counts'] / sum(year_21['counts'])

In [62]:
months_tmp = pd.concat([year_20, year_21], axis = 0).reset_index(drop=True).sort_values(['등록년도', '등록월'])
months_tmp = months_tmp.groupby(['등록월', '분야명'], as_index=False).sum().drop(columns={'등록년도'})

In [127]:
class Polar:
    def __init__(self, data):
        self.data = data
        self.fig = None

    def line(self, r, theta, **kwargs):
        self.fig = px.line_polar(self.data, r=r, theta=theta, **kwargs)

    def bar(self, r, theta, **kwargs):
        self.fig = px.bar_polar(self.data, r=r, theta=theta, **kwargs)

    def scatter(self, r, theta, **kwargs):
        self.fig = px.scatter_polar(self.data, r=r, theta=theta, **kwargs)

    def plot(self):
        self.fig.show()

In [128]:
# data_ = months_tmp
# r = 'counts'
# theta = data_['등록월'].to_numpy() * 30 # degree

# lines = Polar(data_)
# lines.line(r='counts', theta=theta, color='분야명')
# lines.fig.update_traces(fill='toself')
# lines.fig.update_traces(mode='markers+lines')
# lines.fig.update_layout(
#         polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
#         polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
#         #polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
#         title='월별 등록민원 수',
#         title_x=0.5,
#         title_font_size=20
#     )

# lines.plot()

In [129]:
#months = root.groupby(by=["등록월"]).size().reset_index(name="counts").dropna().sort_values('등록월', ascending=False)

# fig = px.line_polar(months_tmp, r='counts', theta=months_tmp['등록월'].to_numpy() * 30, color='분야명')
# fig.update_traces(fill='toself')
# fig.update_traces(mode='markers+lines')

# fig.update_layout(
#     polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
#     polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
#     polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
#     title='월별 등록민원 수',
#     title_x=0.5,
#     title_font_size=20,
# )

# fig.show()

**분야별로 집계된 월별 등록민원 수는 주로 7월, 6월 방향을 향해 뾰족하게 뻗어있는 모양으로 여름에 집중되어있다는 것을 알 수 있음**  

In [66]:
# months = root.groupby(by=["등록월"]).size().reset_index(name="counts").dropna().sort_values('등록월', ascending=False)
# months['percentage'] = round(months['counts'] / sum(months['counts']) * 100, 2)

# fig = px.bar_polar(months, r='counts', theta=months['등록월'].to_numpy() * 30, 
#                        hover_data=['percentage'])

# fig.update_layout(
#     polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
#     polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
#     polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
#     title='월별 등록민원 수',
#     title_x=0.5,
#     title_font_size=20,
# )

# fig.show()

In [67]:
months_ = months_tmp.groupby(['등록월'], as_index=False).sum()

In [68]:
fig = px.scatter_polar(months_, r='counts', theta=months_['등록월'].to_numpy() * 30,) 
                       #hover_data=['percentage'])

fig.update_traces(fill='toself')
fig.update_traces(mode='markers+lines')
fig.update_layout(
    polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
    polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
    polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
    title='월별 등록민원 수',
    title_x=0.5,
    title_font_size=20,
)

fig.show()

**월별 총 민원 수는 여름에 약 55%정도가 집중되어있음**  

### 6. 시간별 민원 수

In [69]:
time = root.groupby(by=["등록시간", "오전오후"]).size().reset_index(name="counts")
time['등록시간_12'] = time['등록시간'].where(time['등록시간'] < 12, time['등록시간'] - 12)
time = time.sort_values('등록시간').dropna().reset_index(drop=True)
time['등록시간'] = time['등록시간'].astype(int)
time['percentage'] = round(time['counts'] / time['counts'].sum() * 100, 2)

In [70]:
fig = (px.line_polar(time, r='counts', theta=time['등록시간_12'].to_numpy() * 30, 
                    color='오전오후', title='시간별 등록민원 수', hover_data=['percentage'])
        )#.for_each_trace(lambda t: t.update(name=t.name.split("=")[1])))

fig.update_traces(fill='toself')
fig.update_traces(mode='markers+lines')

fig.update_layout(
    polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
    polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
    polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
    title='시간별 등록민원 수',
    title_x=0.5,
    title_font_size=20
)

fig.show()

**오전보다 오후에 접수되는 민원 양이 더 많으며**  
**오전은 10시, 11시 오후는 1시 에서 5시 사이에 주로 민원이 접수되는 것을 알 수 있음**

In [71]:
fig = (px.bar_polar(time, r='counts', theta=time['등록시간_12'].to_numpy() * 30, 
                    color='오전오후', title='시간별 등록민원 수', hover_data=['percentage'], barmode='overlay')
        )#.for_each_trace(lambda t: t.update(name=t.name.split("=")[1])))

fig.update_layout(
    polar_angularaxis_tickvals = [hr * 30 % 360  for hr in range(1,13)],
    polar_angularaxis_ticktext = [hr for hr in range(1, 13)],
    polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0)),
    title='시간별 등록민원 수',
    title_x=0.5,
    title_font_size=20
)

fig.show()

**민원이 가장 많이 접수되는 시간은 오전 10시, 11시와 오후 2시, 3시, 4시**  

### 시계열로 본 월별 민원수

In [72]:
root['등록일시_월단위'] = root['등록일시'].dt.strftime("%Y-%m")
df = root.groupby(['등록일시_월단위', '분야명']).size().reset_index(name='counts')

In [73]:
fig = px.line(data_frame=df, x='등록일시_월단위', y='counts',
              color='분야명')
#fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.update_traces(mode='markers+lines')

**2021년 6월, 7월에 여러 분야에서 큰 폭으로 증가하는 것을 볼 수 있음**    

In [74]:
week = root.groupby(['등록요일']).size().reset_index(name='counts')

day_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

bar3 = plot_type(week)
bar3.bar(x='등록요일', y='counts', 
         color='등록요일', category_orders={'등록요일':day_order})
bar3.fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
#bar3.fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
bar3.plot()

**한편 민원 접수는 주말에 큰 폭으로 감소하며, 주중에 주로 접수하는것으로 보임**  
**주말에 민원접수가 낮은 이유는 밖에 잘 나가지 않아서일까?**  

In [75]:
root.loc[:,['제목','질문내용','답변내용']] = root.loc[:,['제목','질문내용','답변내용']].apply(lambda x:x.str.replace(r'[(\n)(\t)(&quaot;)]', '', regex=True))
root[['제목', '질문내용', '답변내용']] = root[['제목', '질문내용', '답변내용']].apply(lambda x: x.str.replace(r'[^a-z|가-힣|0-9| ]', '', regex=True))

In [39]:
with open(dir + '/Data/minwon_ver3.pkl' , 'wb') as f:
    pkl.dump(root, f)

NameError: ignored

In [ ]:
class Dictionary:
    def __init__(self, base, params=None):
        self.base = base
        self.model =  LRNounExtractor_v2(verbose=False)
        self.output = None
        self.noun = None
        if params != None:
            self.params = params
        else:
            self.params = {
                'min_noun_score' : 0.7, # 최소 스코어
                'min_noun_frequency' : 5 # 최수 빈도수
            }

    def construct(self):
        self.output = self.model.train_extract(self.base,
                                               **self.params)

    def build(self):
        Dictionary.construct(self)
        # 1글자인 단어 제거
        self.noun = [(k, v.score) for k, v in self.output.items() if (len(k) > 1)]
        return self.noun

In [ ]:
title   = root['제목']
content = root['질문내용']
dic = Dictionary(content).build() #문서 기반 사전 생성

In [ ]:
# 사용자 사전 추가
# dic.extend([('생활불편신고', 1.0), ('장애인주차구역', 1.0), ('장애인전용구역', 1.0), ('장애인구역', 1.0), ('공동주택', 1.0),
#             ('불법주차', 1.0), ('청탁금지법', 1.0), ('장애인전용주차구역', 1.0), ('중앙분리대', 1.0), ('아이스팩', 1.0),
#             ('해양수산직', 1.0)])

In [ ]:
class Tokenizer:
    def __init__(self, data):
        self.data = data
        self.kiwi = Kiwi()
        self.output = []

    def add_dictionary(self, dic):
        for i in dic:
            self.kiwi.add_user_word(i[0], 'NNP', i[1] + 1)
    
    def fit(self, passtag=None):
        if passtag == 'Noun':
            for result in self.kiwi.analyze(self.data):
                tokens, score = result[0]
                tmp = [word.form for word in tokens if ('NN' in word.tag) and (len(word.tag) > 1)]
                self.output.append(tmp)
        # else:
        #     for result in self.kiwi.analyze(self.data):
        #         tokens, score = result[0]
        #         tmp = [word.form + '다' if word.tag in ['VV', 'VA'] else word.form for word in tokens if (word.tag in ['NNP', 'NNG', 'MAG', 'VA', 'VV', 'MM']) and (len(word.tag) > 1)]
        #         self.output.append(tmp)

        return self.output

In [ ]:
tokenizer = Tokenizer(content)
tokenizer.add_dictionary(dic)
tokens = tokenizer.fit(passtag='Noun')
#tokens_all = tokenizer.fit()

In [ ]:
stopwords = ['안녕하세요', '우선', '아니고', '어떻게', '요청', '요망', '가요', '있나요', '어떠'
             '문의', '여부', '가능', '관련', '질의', '방법', '건의', '요청', '필요', '질문',
             '예정', '하여', '어느', '부탁', '궁금', '있는지', '무엇', '대하여', '알고', '궁금',
             '바랍니다', '확인', '어디', '어디서', '충청북도', '충주시', '대구', '중구', '이해',
             '요건']

In [ ]:
tokens = pd.Series(tokens).map(lambda x:[w for w in x if (not w in stopwords) and (len(w) > 1)]).to_list()

In [ ]:
root['tokens'] = tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=5,
                             max_df=0.8,
                             ngram_range=(2,4),
                             stop_words=stopwords
                             )

In [ ]:
dep = {
    '교육/문화/체육/관광' : root.loc[root['분야명'] == '교육/문화/체육/관광', 'tokens'],
    '국토/교통/농림/해양' : root.loc[root['분야명'] == '국토/교통/농림/해양', 'tokens'],
    '재정/금융/소비자' : root.loc[root['분야명'] == '재정/금융/소비자', 'tokens']
}

In [ ]:
for key in dep.keys():
    vector = vectorizer.fit_transform(pd.Series(dep[key]).astype(str))
    scores = vector.toarray().sum(axis=0)
    idx = np.argsort(-scores)
    scores = scores[idx]
    vocab = np.array(vectorizer.get_feature_names())[idx]
    dep[key] = pd.DataFrame(list(zip(vocab, scores))[:30],
                            columns=['단어', '빈도수기반점수'])

In [ ]:
dep[key] = pd.DataFrame(list(zip(vocab, scores))[:20],
                            columns=['단어', '빈도수기반점수'])

In [ ]:
dep['국토/교통/농림/해양']['분야명'] = '국토/교통/농림/해양'
dep['교육/문화/체육/관광']['분야명'] = '교육/문화/체육/관광'
dep['재정/금융/소비자']['분야명'] = '재정/금융/소비자'

In [ ]:
df = pd.concat(dep.values(), ignore_index=True)

In [ ]:
fig = px.bar(dep['재정/금융/소비자'], x='빈도수기반점수', y='단어', facet_col='분야명', color='분야명')
fig.update_layout(yaxis_categoryorder='total descending')
fig.show()

In [ ]:
fig = px.treemap(df, path=['분야명', '단어'], values='빈도수기반점수',
                 color='분야명', title='분야별 민원 Treemap')
fig.show()

**교육/문화/체육/관광 분야에서는 졸업학교(분교장) 폐교와 관련한 문의, 방과후학교 관련 문의, 학교폭력, 교육비 및 학자금 관련 문의가 주요 안건**  
**재정/금융/소비자 분야에서는 연말정산, 현금영수증, 종합소득세, 사업자등록, 근로장려금 관련 문의가 주요 안건**  
**국토/교통/농림/해양 분야에서는 주로 해양관련이 대부분이며, 어업경영체, 해기사면허, 항만시설, 선박 해양쓰레기 관련 문의가 주요 안건**  

교육/문화/체육/관광 분야에서는 학교폭력이라는 키워드가 어느정도 중요하다고 보이며,  
재정/금융/소비자 분야에서는 근로장려금, 연말정산이라는 키워드가 중요하다고 생각되며,  
국토/교통/농림/해양 분야에서는 주로 해양관련 키워드가 주를 이룬다고 보인다.  

위 시계열 그래프에서도 재정/금융/소비자 분야에서 2021년 5월, 6월에 피크가 나타났었는데 (2020년은 데이터가 없어 확인 불가) 이는 근로장려금 신청관련이라고 생각되며, 2021년 1월 역시 작은 피크가 있어 이는 연말정산 관련이라고 생각된다.



이 중에서 눈여겨보아야 할 부분이 연말정산으로 올해가 얼마 남지 않았기 때문에 시민들의 관심 대상이 될 것이라 생각  
따라서 연말정산 관련 문의가 무엇인지 세부적으로 파악해보려고 함

In [ ]:
year_tax = root.loc[(root['분야명']=='재정/금융/소비자') & (root['질문내용'].str.contains('연말정산')), '질문내용']
tokenizer = Tokenizer(year_tax)
tokenizer.add_dictionary(dic)
tokens_ = tokenizer.fit(passtag='Noun')
#tokens_all = tokenizer.fit()

In [ ]:
pd.Series(tokens_).sample(5)

49     [년, 월, 일, 현금, 영수증, 미발급, 대하여, 홈텍스, 신고, 지금, 처리, ...
97     [연말, 정산, 시, 공제, 항목, 것, 연말, 정산, 소득공제, 내용, 정정, 추...
118    [저희, 회사, 연말, 정산, 대상, 근로자, 입사, 지역, 가입자, 납부, 국민,...
120          [근로소득, 간이지급명세서, 제출, 연말, 정산, 지급, 명세서, 등, 제출]
37       [안녕하세요제, 음료, 배달, 중, 근로자, 사업소득, 이것, 연말, 정산, 있나요]
dtype: object

In [ ]:
year_tax.sample(10).to_list()

['제가 오늘 순천세무서로부터 2019년 귀속 종합소득세를 무신고하여 과세자료가 발생하였으니 홈택스 등을 이용하여 기한후신고를 하라고안내를 받았습니다제가 알기로는 근로소득자는 연말정산을 통해 환급받거나 추가납부하면 끝나며 종합소득세 신고대상은 아닌것으로 알고 있습니다저는 2014년부터 순천에 소재한 중소기업에서 근로소득을 지급받고 있는데 확인해 보니 여수에 소재한 처음듣는 법인업체에서근로소득을 지급한것으로 신고를 한것이었습니다 인터넷에 해당 사업장에 전화를 걸어 어떻게 된 영문인지 확인해보니주민등록번호를 잘못 입력해서 신고한것이었습니다 해당 사업장 관계자는 주민등록번호 오기입에 따라 잘못 신고가 된거라고 다시 수정제출한다고 했습니다 앞으로는 이런일이 절대 일어나지 않았으면 좋겠고 이 사업장에는 세무서에서 어떤 불이익이 주는지 알고 싶습니다',
 '안녕하십니까 회사를 다니고 있는 직원으로서 지난 2020년 근로소득 연말정산에서 월세액 세액공제가 누락한 사실을 확인하였습니다  준비서류는 확인하였는데  누락한 월세액 세액공제신청시  현금영수증 등 소득공제신청도 동시에 가능한지 문의합니다',
 '연말정산자료를 제출했는데 회사에서 입력하면서 부모님공제를 누락하였습니다 사업자는 다시신고하는 절차가 번거롭고 사실상 힘들다고 하면서 소득세 신고때 개인이 직접할 수 있다고 합니다 5월에 소득세 신고시 직접 신고해서 추가할 수 있을까요',
 '연말정산이란 무엇인가요',
 '제가 20년도 월세 납부내역 현금영수증 신청을 하였는데 남인천세무서측에서 신고내용과 다르게 처리가 되어서 대표 전화번호로 전화를 수십번을 했는데 연결이 안되네요홈텍스 고객센터랑 통화하니까 남인천세무서측에서 처리한거라서 남인천세무서에 연락을 해보라는데 몇주동안 전화연결 자체가 안되네요코로나때문에 방문 자제하라면서 직접 찾아가야만 처리가 되는건지 대표전화 써놓고 전화연결이 안되는건 이해가 안되네요 제가 19년도 연말정산 당시 사업자와 25만원에 계약한 건에 대해 현금영수증 신청한 이력이 있으나 당시 착오가 있어 

In [ ]:
stopwords = ['있나요', '하나요', '있음', '받는것', '경우', '하여',
             '안녕하세요', '가요', '안녕하세요제']

In [ ]:
tokens2 = pd.Series(tokens_).map(lambda x:[w for w in x if (not w in stopwords) and (len(w) > 1)]).to_list()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=5,
                             max_df=0.8,
                             ngram_range=(2,3),
                             stop_words=stopwords
                             )

vec_tokens = vectorizer.fit_transform(pd.Series(tokens2).astype("U"))

In [ ]:
vectorizer.get_feature_names()

['근로소득 연말',
 '근로소득 연말 정산',
 '부양가족 공제',
 '소득공제 누락',
 '연말 정산 간소화',
 '연말 정산 결과',
 '연말 정산 공제',
 '연말 정산 누락',
 '연말 정산 소득공제',
 '장기주택저당차입금 이자상환액',
 '정산 간소화',
 '정산 결과',
 '정산 공제',
 '정산 누락',
 '정산 소득공제',
 '종합소득세 신고',
 '필요 서류',
 '현금 영수증',
 '회사 연말',
 '회사 연말 정산']

In [1]:
!pwd

/content


In [ ]:
%%shell
jupyter nbconvert --to html /Your notebook path/file.ipynb